# Yahoo Japan Auction Scraper

Do you want to browse japanese auction sites with ease?

Are you fed up of seeing 
![](https://i2.wp.com/re-how.net/wp-content/uploads/2019/09/d17182-163-128563-1.png?resize=605%2C208&ssl=1)

When u see this, are you like:
![](https://i.kym-cdn.com/photos/images/original/000/997/001/d1b.png)

Have i got the solution for you.

## Welcome to **Leo's very basic yahoo japan auction scraper**
(ps. doesnt actually scrape yahoo japan auctions eheheh)

I will guide you thru on finding your grails using my scraper

In [1]:
# Run all of these other wise it wont work ! !

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from IPython.display import Image, HTML
pd.set_option('display.max_colwidth', -1)

In [2]:
def currency_converter(cfrom, cto, api_key):
    """
    Your one stop function at getting the most up to date (maybe) currency conversion rate
    """
    url = 'https://free.currconv.com/api/v7/convert?q={}_{}&compact=ultra&apiKey={}'.format(cfrom, cto, api_key)
    r_ = requests.get(url)
    return r_.json()

In [3]:
def to_new_currency(x, c_rate):
    """
    Calculating the price in the currecny you desire ! !
    """
    return round(float(x.split()[0].replace(',','')) * c_rate,2)

In [9]:
def buyee_scrape(brand, end_page, c_rate, search_type): # order | to add in
    """
    LET'S GET SCRAPING
    
    Arguments:
    brand: string, what brand are you looking for
    page: int, how many pages do you want to scrape 
    c_rate: float, the current conversion rate for your desired currency
    search_type: int, choose type from list - [0 - newly listed, 1 - ending soon, 2 -lowest price,
                                                3 - highest price, 4 - most popular highest ranked]
    """
    brand = brand.replace(' ','%20').lower() 
    
    # where i store the original link
    yahoo_jjp_link = pd.Series()
    # where i store the picture link
    pic = pd.Series()
    # where i store the current price (bidding price may differ from buyout)
    price = pd.Series()
    
    search_list = ['new_listed','ending_soon','lowest_price','highest_price','most_pop_new']
    
    search_dict = {'new_listed':'end&order=d',
     'ending_soon':'end&order=a',
     'lowest_price':'cbids&order=a',
     'highest_price':'cbids&order=d',
     'most_pop_new':'score&ranking=popular'
    }
    
    search = search_dict[search_list[search_type]]
    
    #https://buyee.jp/item/search/query/comme?sort=score&ranking=popular&translationType=1&page=2
    #https://buyee.jp/item/search/query/comme?sort=score&ranking=popular&translationType=1&page=3
    
    for page in range(1,(end_page+1)):
        buyee ='https://buyee.jp/item/search/query/{}?sort={}&translationType=1&page={}'.format(brand, search, page)
        #print(buyee)
        # grab them pages
        buyee = requests.get(buyee)
        buyeesoup = BeautifulSoup(buyee.text, 'html.parser')


        for product_all in buyeesoup.find_all('ul', attrs={'class':'auctionSearchResult list_layout'}):
            for item in product_all.find_all('li',attrs={'class':'itemCard'}):

                # Append yh jpn auction link
                try:          
                    href = ('https://page.auctions.yahoo.co.jp/jp/auction/'+item.find('a')['href'].split('/')[-1])
                    yahoo_jjp_link.at[href] = href
                except:
                    pass

                # Append item pic link -- maybe scrape pic later
                try:
                    pic.at[href] = item.find('img',attrs={'class':'hide g-thumbnail__image'})['data-src']
                except:
                    pass

                # scrape price -- it repeats three time before showing both current and buy out

                price.at[href] = item.find('span',attrs={'class':'g-price'}).text

    yjp = pd.concat([yahoo_jjp_link,pic, price], axis=1).reset_index(drop=True)
    yjp.columns = ['yjpn_link', 'pic_link', 'price']
    yjp.price = yjp.price.apply(lambda x: to_new_currency(x,c_rate))

    return yjp




In [5]:
def path_to_image_html(path):
    """
    Allows html image links to be seen within pandas
    """
    return '<img src="'+ path + '"/>'

In [6]:
# run this to get the current conversion rate 
c_rate = currency_converter(cfrom= 'JPY',cto='GBP',api_key='40c8cabfa8e1588ce050')['JPY_GBP']

In [7]:
# lets have a peak at that conversion rate
print('its a whopping ',c_rate,'yen to 1 pound')

its a whopping  0.007551 yen to 1 pound


In [10]:
# lets check out comme des garcon
scrape = buyee_scrape('comme des garcon', 10, c_rate, 0)

In [11]:
# woah look at this amazing dataframe with so much information looooool.
scrape

,yjpn_link,pic_link,price
0,https://page.auctions.yahoo.co.jp/jp/auction/v551916978,https://wing-auctions.c.yimg.jp/sim?furl=auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0405/users/5aab008bffca13c322ad4deb69bfd05b43aa2f6e/i-img1200x1200-1525318396wxslnb19094.jpg&dc=1&sr.fs=20000,98.16
1,https://page.auctions.yahoo.co.jp/jp/auction/d424433152,https://wing-auctions.c.yimg.jp/sim?furl=auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0103/users/047eec4caba24236d792d0c9d533b19e627af18a/i-img640x960-1583566360frqscb278293.jpg&dc=1&sr.fs=20000,264.29
2,https://page.auctions.yahoo.co.jp/jp/auction/h404901268,https://wing-auctions.c.yimg.jp/sim?furl=auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0506/users/0d1ebdd8d3cb79f9033fbc122b0f338feca277f2/i-img427x640-1561273299a5jjip242614.jpg&dc=1&sr.fs=20000,98.16
3,https://page.auctions.yahoo.co.jp/jp/auction/u278090202,https://wing-auctions.c.yimg.jp/sim?furl=auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0505/users/08ca4dd88f0d9bdbe5d83281a0b2323f9855ddb7/i-img480x717-15580811807bfzhh720659.jpg&dc=1&sr.fs=20000,22.65
4,https://page.auctions.yahoo.co.jp/jp/auction/r364433987,https://wing-auctions.c.yimg.jp/sim?furl=auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0511/users/ae54e41e78981878213a34441f073fdd0142df1d/i-img321x477-1574847053ylvfnp119430.jpg&dc=1&sr.fs=20000,113.27
5,https://page.auctions.yahoo.co.jp/jp/auction/b412090243,https://wing-auctions.c.yimg.jp/sim?furl=auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0508/users/ecf12bbd8dc5d67db125447f72aa52b0bc231b1f/i-img750x750-1567182147gc6iq6120770.jpg&dc=1&sr.fs=20000,143.46
6,https://page.auctions.yahoo.co.jp/jp/auction/g305735103,https://wing-auctions.c.yimg.jp/sim?furl=auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0411/users/c4bf01b8e97dab4685d17860395850b47456d5fc/i-img1200x900-1541322006zfe7vh1812826.jpg&dc=1&sr.fs=20000,90.60
7,https://page.auctions.yahoo.co.jp/jp/auction/b430984991,https://wing-auctions.c.yimg.jp/sim?furl=auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0511/users/93e8064e34cd67146c36050e294c3e782b94ab88/i-img900x1200-1574664067hrkuc071699.jpg&dc=1&sr.fs=20000,106.32
8,https://page.auctions.yahoo.co.jp/jp/auction/r282307718,https://wing-auctions.c.yimg.jp/sim?furl=auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0411/users/c4bf01b8e97dab4685d17860395850b47456d5fc/i-img1200x900-1541321767gxs9171981303.jpg&dc=1&sr.fs=20000,75.49
9,https://page.auctions.yahoo.co.jp/jp/auction/c786399020,https://wing-auctions.c.yimg.jp/sim?furl=auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0510/users/10d777fc67408640e77d3544277ab616a572cd97/i-img233x312-15703035835xkhlu198760.jpg&dc=1&sr.fs=20000,135.92


In [12]:
HTML(scrape.to_html(escape=False ,formatters=dict(pic_link=path_to_image_html)))

,yjpn_link,pic_link,price
0,https://page.auctions.yahoo.co.jp/jp/auction/v551916978,,98.16
1,https://page.auctions.yahoo.co.jp/jp/auction/d424433152,,264.29
2,https://page.auctions.yahoo.co.jp/jp/auction/h404901268,,98.16
3,https://page.auctions.yahoo.co.jp/jp/auction/u278090202,,22.65
4,https://page.auctions.yahoo.co.jp/jp/auction/r364433987,,113.27
5,https://page.auctions.yahoo.co.jp/jp/auction/b412090243,,143.46
6,https://page.auctions.yahoo.co.jp/jp/auction/g305735103,,90.60
7,https://page.auctions.yahoo.co.jp/jp/auction/b430984991,,106.32
8,https://page.auctions.yahoo.co.jp/jp/auction/r282307718,,75.49
9,https://page.auctions.yahoo.co.jp/jp/auction/c786399020,,135.92
